In [2]:
from PIL import Image
import os
import numpy as np
import tensorflow as tf
import pandas as pd

In [3]:
IMAGE_SIZES = [100, 100]

In [8]:
# reads all images in filepath, resizes it according with 2D list img_sizes and 
# store them in a np_array. The string filepath can only contain images (no other
# file types such as txt for example are allowed). 
# It works only for RGB images.
def images_to_np_array(filepath, img_sizes):
    """
    This function reads all imagens in a given folder, resizes them and stores them in a numpy array.

    Parameters:
    filepath (str): The path to the folder containing the images.
    img_sizes (list): A list containing the desired height and width of the images.

    Returns:
    np.ndarray: A numpy array containing all the images resized to the specified dimensions.
    """
    
    list_of_files = []

    for dirpath, _, filenames in os.walk(filepath):
        for f in filenames:
            full_path = os.path.join(dirpath, f)
            list_of_files.append(full_path)

    # number of image files
    n_images = len(list_of_files)

    # create np_array to store all images
    array_images = np.zeros((n_images, img_sizes[1], img_sizes[0], 3), dtype=np.uint8)

    for index, image_path in enumerate(list_of_files):
        with Image.open(image_path) as img:
            img_rgb = img.convert('RGB')
            img_resized = img_rgb.resize(img_sizes)
            array_images[index] = np.asarray(img_resized)

    return array_images



In [9]:
# read all capivaras
capivara_array = images_to_np_array(filepath = "./capivara", img_sizes = IMAGE_SIZES)
y_capivara = np.zeros(capivara_array.shape[0])

# read all dunkies
burro_array = images_to_np_array(filepath = "./burro", img_sizes = IMAGE_SIZES)
y_burro = np.ones(burro_array.shape[0])

# create y vector with labels
y = np.concatenate( (y_capivara, y_burro) )

# concatenate and shuffle images
array_images = np.concatenate( (capivara_array, burro_array), axis = 0 )
number_of_images = array_images.shape[0]
permuted_index = np.random.permutation(number_of_images)
array_images = array_images[permuted_index, :, :, :]
y = y[permuted_index]

# splitting training and test sets

x_train = array_images[0:700,:,:,:]
y_train = y[0:700]

x_test = array_images[700:,:,:,:]
y_test = y[700:]


In [10]:
# preprocessing 
array_images = tf.keras.applications.densenet.preprocess_input(array_images, data_format = "channels_last")

In [11]:

# initialize model with weights treined in imagenet 
base_model = tf.keras.applications.DenseNet169(
    include_top=False,
    weights="imagenet",
    input_shape=(IMAGE_SIZES[0], IMAGE_SIZES[1], 3),
    pooling="avg",
    classes=2,
    classifier_activation="None"
)

# Freeze model weights
base_model.trainable = False

51877672/51877672 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step


In [12]:
data_augmentation = tf.keras.Sequential(
    [tf.keras.layers.RandomFlip("horizontal"), tf.keras.layers.RandomRotation(0.5),]
)

In [13]:
inputs = tf.keras.Input(shape=(IMAGE_SIZES[0], IMAGE_SIZES[1], 3))
# apply data augmentation
x = data_augmentation(inputs)
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base_model(x, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = tf.keras.layers.Flatten(data_format = "channels_last")(x)
x = tf.keras.layers.Dense(100, activation = "relu")(x)
x = tf.keras.layers.Dense(100, activation = "relu")(x)
x = tf.keras.layers.Dense(100, activation = "relu")(x)
outputs = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(inputs, outputs)

model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet169 (Functional)        │ (None, 1664)           │    12,642,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1664)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │       166,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,829,681 (48.94 MB)

 Trainable params: 186,801 (729.69 KB)

 Non-trainable params: 12,642,880 (48.23 MB)

In [18]:
#checkpoint_filepath = "./trained_model_checkpoint"

# define the callback (to save the weights that gave best val_accuracy during training)
#callback = tf.keras.callbacks.ModelCheckpoint(monitor='val_accuracy',
#                                              mode = "max",
#                                              save_best_only = True,
#                                              filepath = checkpoint_filepath)


In [14]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

history = model.fit(x = x_train, 
                    y = y_train, 
                    epochs = 10, 
                    validation_data = (x_test, y_test), 
                    batch_size = 100)


Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step - binary_accuracy: 0.3923 - loss: 2.3855 - val_binary_accuracy: 0.6852 - val_loss: 1.5290
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - binary_accuracy: 0.6207 - loss: 1.3231 - val_binary_accuracy: 0.6667 - val_loss: 0.9537
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - binary_accuracy: 0.6064 - loss: 0.9005 - val_binary_accuracy: 0.5556 - val_loss: 0.9791
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - binary_accuracy: 0.6089 - loss: 0.8876 - val_binary_accuracy: 0.7037 - val_loss: 0.6562
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - binary_accuracy: 0.6659 - loss: 0.6999 - val_binary_accuracy: 0.7315 - val_loss: 0.5997
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 1s/step - binary_accuracy: 0.6734 - loss: 0.6720 - val_binary_accuracy: 0.7130 - val_loss: 0.6694
Epoch 7/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - binary_accuracy: 0.7245 - loss: 0.6112 - val_binary_accuracy: 0.7685 - val_loss: 0.5609
Epoch 8/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 

In [15]:
model.evaluate(y = y_test, x = x_test, verbose = 2)

4/4 - 2s - 565ms/step - binary_accuracy: 0.7778 - loss: 0.5123


[0.5122657418251038, 0.7777777910232544]

In [16]:
predictions = model.predict(x_test)
pd.DataFrame( {"probability": tf.keras.activations.sigmoid(predictions)[:,0], "label": y_test})

4/4 ━━━━━━━━━━━━━━━━━━━━ 19s 3s/step 


,probability,label
0,0.096672,0.0
1,0.249293,0.0
2,0.918419,0.0
3,0.208197,0.0
4,0.281596,1.0
...,...,...
103,0.266887,0.0
104,0.400155,0.0
105,0.324594,1.0
106,0.245146,0.0


In [17]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
base_model.trainable = True
model.summary()

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),  # Low learning rate
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.BinaryAccuracy()],
)

epochs = 10
model.fit(x = x_train, y = y_train, epochs=epochs, validation_data = (x_test, y_test))

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 100, 100, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ densenet169 (Functional)        │ (None, 1664)           │    12,642,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1664)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │       166,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        10,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           101 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,203,285 (50.37 MB)

 Trainable params: 12,671,281 (48.34 MB)

 Non-trainable params: 158,400 (618.75 KB)

 Optimizer params: 373,604 (1.43 MB)

Epoch 1/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - binary_accuracy: 0.6079 - loss: 0.6724 - val_binary_accuracy: 0.7130 - val_loss: 0.4340
Epoch 2/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - binary_accuracy: 0.6235 - loss: 0.5957 - val_binary_accuracy: 0.7130 - val_loss: 0.3951
Epoch 3/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - binary_accuracy: 0.6388 - loss: 0.5739 - val_binary_accuracy: 0.7222 - val_loss: 0.3611
Epoch 4/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - binary_accuracy: 0.5785 - loss: 0.5640 - val_binary_accuracy: 0.7315 - val_loss: 0.3321
Epoch 5/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 35s 2s/step - binary_accuracy: 0.6237 - loss: 0.5127 - val_binary_accuracy: 0.7870 - val_loss: 0.3192
Epoch 6/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 34s 2s/step - binary_accuracy: 0.6679 - loss: 0.4650 - val_binary_accuracy: 0.8056 - val_loss: 0.3052
Epoch 7/10
22/22 ━━━━━━━━━━━━━━━━━━━━ 33s 2s/step - binary_accuracy: 0.6742 - loss: 0.4306 - val_binary_accuracy: 0.8148 - val_loss: 0.2864
Epoch 8/10
22/22 ━━